In [5]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
#importing the data set
dataset = pd.read_csv('ML-EdgeIIoT-dataset.csv')
X = dataset.iloc[:,:-1].values
Y = dataset.iloc[:,-1].values

C:\Users\USER\AppData\Local\Temp\ipykernel_18040\3850422953.py:2: DtypeWarning: Columns (3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('ML-EdgeIIoT-dataset.csv')


In [ ]:
# Fixed notebook formatting